# Exercise 4a - ZKP Considerations - measurements

The main difference we saw, so far, is that LegoGroth16 requires the verifier to create a key pair, and send one key to the credential holder, while using the other one later to verify the range proof.

**Now, we will compare the two algorithms on two aspects:**
1. Time to create the proof
2. Size of messages exchanged

**Note:**
We will use **BP** for bulletProof++ and **LG** for LegoGroth16

## Sections

1. Setup
2. LegoGroth16
3. Bulletproofs++
4. Printing values for plots

---

## 1. Setup

In [ ]:
// Common setup - creating a credential, an issuer, and a signature.

import { initializeWasm, BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES,
       CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

// Credential used for measurements
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0, maximum: 120},
        height: {type: 'integer', minimum: 130, maximum: 210},
        weight: {type: 'number', minimum: 40, multipleOf: 0.1}
    }
}
const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    profession: "IT Manager",
    height: 176,
    weight: 84
}

// Setting up the issuer and creating the signatures

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

const schema = CredentialSchema.essential(); 
schema.properties[SUBJECT_STR] = E_ID_SCHEMA

const builder = new BBSCredentialBuilder();
builder.schema = new CredentialSchema(schema);
builder.subject = E_ID_DATA

const credential = builder.sign(secretKey);

const credentialToSendToHolder = credential.toJSON();

---

## 2. LegoGroth16

In [ ]:
// LegoGroth16
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';

// We measure the time taken to create the verification keys
let LGProofTimes: number[] = []
let LGProofSizes: number[] = []
let LGVerificationTimes: number[] = []

// Setup time
let startTime = performance.now();
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

let LGSetupTime = Math.floor(performance.now() - startTime) / 1000;
LGProofTimes.push(LGSetupTime);
console.log(`Time taken to setup LG16: ${LGSetupTime} seconds`);

// Proof creation without any range bounds
startTime = performance.now()
let builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
let presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, undefined);
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
LGProofTimes.push(consumedTime);
LGVerificationTimes.push(verificationTime);
LGProofSizes.push(presentation.proof.bytes.length);                
console.log(`Proof size without any range bounds is: ${presentation.proof.bytes.length} bytes`);
console.log(`Time taken to generate proof with no range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with one range bounds 
startTime = performance.now()
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(), new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', snarkProvingKey);
presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', snarkVerifyingKey]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
LGProofTimes.push(consumedTime);
LGVerificationTimes.push(verificationTime);
LGProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with one range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with one range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with two range bounds 
startTime = performance.now()
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(), new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', snarkProvingKey);
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', snarkProvingKey);
presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', snarkVerifyingKey], ['heightRangeCheck', snarkVerifyingKey]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
LGProofTimes.push(consumedTime);
LGVerificationTimes.push(verificationTime);
LGProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with two range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with two range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with three range bounds 
startTime = performance.now();
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey);
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(), new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', snarkProvingKey);
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', snarkProvingKey);
builder.enforceBounds(0, 'credentialSubject.weight', 70, 85, 'weightRangeCheck', snarkProvingKey);

presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', snarkVerifyingKey], ['heightRangeCheck', snarkVerifyingKey], ['weightRangeCheck', snarkVerifyingKey]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
LGProofTimes.push(consumedTime);
LGVerificationTimes.push(verificationTime);
LGProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with three range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with three range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

---

## 3. Bulletproofs++

In [ ]:
// Bulletproofs++
import { BoundCheckBppParams, PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// We measure the time taken to create the verification keys
let BPProofTimes: number[] = []
let BPProofSizes: number[] = []
let BPVerificationTimes: number[] = []
// BulletProof++ -- setup time
let startTime = performance.now();

const boundCheckBppParams = new BoundCheckBppParams(stringToBytes('Bulletproofs++ testing')).decompress();
const BPSetupTime = Math.floor(performance.now() - startTime) / 1000;

BPProofTimes.push(BPSetupTime)
console.log(`Time taken to setup BP++: ${BPSetupTime} seconds`)

// Proof creation without any range bounds
startTime = performance.now()

let builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
let presentation = builder.finalize();

let consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, undefined)
let verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
BPProofTimes.push(consumedTime);
BPVerificationTimes.push(verificationTime);
BPProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size without any range bounds is: ${presentation.proof.bytes.length} bytes`);
console.log(`Time taken to generate proof with no range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with one range bounds 
startTime = performance.now()
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(
    0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(),
    new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', boundCheckBppParams
);
presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', boundCheckBppParams]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
BPProofTimes.push(consumedTime);
BPVerificationTimes.push(verificationTime);
BPProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with one range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with one range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with two range bounds 
startTime = performance.now()
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(
    0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(),
    new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', boundCheckBppParams
);
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', boundCheckBppParams);
presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', boundCheckBppParams], ['heightRangeCheck', boundCheckBppParams]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
BPProofTimes.push(consumedTime);
BPVerificationTimes.push(verificationTime);
BPProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with two range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with two range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

// Proof creation with three range bounds 
startTime = performance.now()
builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
builder.enforceBounds(0, 'credentialSubject.timeOfBirth', new Date("1972-01-01T00:00:00").getTime(), new Date("2020-08-01T00:00:00").getTime(), 'timeOfBirthRangeCheck', boundCheckBppParams);
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', boundCheckBppParams);
builder.enforceBounds(0, 'credentialSubject.weight', 70, 85, 'weightRangeCheck', boundCheckBppParams);
presentation = builder.finalize();
consumedTime = Math.floor(performance.now() - startTime) / 1000;
presentation.verify([publicKey], undefined, new Map([['timeOfBirthRangeCheck', boundCheckBppParams], ['heightRangeCheck', boundCheckBppParams], ['weightRangeCheck', boundCheckBppParams]]));
verificationTime = (Math.floor(performance.now() - startTime - (consumedTime * 1000)) / 1000);
BPProofTimes.push(consumedTime);
BPVerificationTimes.push(verificationTime);
BPProofSizes.push(presentation.proof.bytes.length);
console.log(`Proof size with three range bounds is: ${presentation.proof.bytes.length}  bytes`);
console.log(`Time taken to generate proof with three range bounds ~${consumedTime} seconds`);
console.log(`Time taken to validate proof with no range bounds ~${verificationTime} seconds`);

---

## 4. Printing values for plots

In [ ]:
console.log(BPProofTimes, LGProofTimes, BPProofSizes, LGProofSizes, BPVerificationTimes, LGVerificationTimes);
console.log(provingKey.value.length);